In [ ]:
import os
from google import genai
from tqdm.auto import tqdm
from dotenv import load_dotenv
from time import sleep\
    
load_dotenv()

In [ ]:
implementer = "Nam"
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key = GEMINI_API_KEY)
model = "gemini-2.0-flash"

In [ ]:
def build_prompt(ques_list):
    instructions = """
    Bạn sẽ nhận được tối đa 10 câu hỏi. Hãy phân loại từng câu hỏi là "text" hoặc là "non-text".
    
    **Định nghĩa**:
    - "text": bao gồm các câu hỏi dựa trên các yếu tố trích xuất từ văn bản, liên quan đến các số liệu, từ ngữ, hoặc bất kỳ hình thức văn bản nào.
    - "non-text": bao gồm các câu hỏi không dựa trên các yếu tố văn bản, chẳng hạn như các câu hỏi dựa trên hình ảnh, vật thể, màu sắc, hình dạng, v.v.
    
    Trả lời đầu ra là một danh sách gồm đúng N dòng, với mỗi dòng chỉ là một từ: "text" hoặc "non-text", tương ứng với từng câu hỏi theo thứ tự.
    
    Dưới đây là các câu hỏi cần phân loại:
    """.strip()
    
    formatted_questions = "\n".join(f"{i}. {q}" for i, q in enumerate(ques_list, 1))

    ending = "\n\nHãy trả về danh sách 1 từ duy nhất cho mỗi dòng: \"text\" hoặc \"non-text\"."

    full_prompt = f"{instructions}\n{formatted_questions}{ending}"
    return full_prompt

In [ ]:
def prep_ques(file_path: str):
    ques = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(" ||| ")
            if len(parts) != 2:
                raise
            ques.append((parts[0], parts[1]))
    return ques

In [ ]:
batch_size = 10
store_file = f"{implementer}_result.txt"
ques = prep_ques(f"{implementer}_ques.txt")


start_index = 0
if os.path.isfile("index.txt"):
    with open(f"index.txt", "r", encoding="utf-8") as f:
        start_index = int(f.read().strip())
print(f"Starting from index {start_index}")


api_error = False
batch_ques, batch_typ = [], []
with tqdm(total = len(ques), dynamic_ncols=True) as pbar:
    try:
        for idx in range(len(ques)):  
            if idx < start_index:
                pbar.update(1)
                continue
            batch_ques.append(ques[idx][0])
            batch_typ.append(ques[idx][1])
            
            if len(batch_ques) == batch_size or idx == len(ques) - 1:
                prompt = build_prompt(batch_ques)
                cnt = 0
                while True:
                    try:
                        response = client.models.generate_content(
                            model="gemini-2.0-flash", 
                            contents=prompt,
                        ).text.strip().split('\n')
                        break
                        
                    except Exception as e:
                        if "429" in str(e):
                            if cnt > 8:
                                api_error = True
                                break
                            cnt += 1
                            print(f"Re-trying in 10s")
                            sleep(10)
                if api_error:
                    raise Exception("API Error: 429 Too Many Requests")
                
                with open(store_file, "a", encoding="utf-8") as f:
                    for i in range(len(batch_ques)):
                        f.write(f"{batch_ques[i]} ||| {batch_typ[i]} ||| {response[i]}\n")
                with open(f"index.txt", "w", encoding="utf-8") as f:
                    f.write(f"{idx + 1}")
                
                pbar.update(len(batch_ques))
                batch_ques.clear()
                batch_typ.clear()
                
    except KeyboardInterrupt as k:
        print(f"Interrupted")
    except Exception as e:
        if api_error:
            print(f"API Error: 429 Too Many Requests")
        else:
            print(f"An error occurred: {e}")
    finally:
        print(f"Done") 

In [2]:
def check_accu(listFiles: list[str]):
    total, right = 0, 0                
    for file in listFiles:
        with open(file, "r", encoding = "utf-8") as f:
            for line in f:
                parts = line.strip().split(" ||| ")
                if len(parts) != 3:
                    raise
                total += 1
                if parts[1] == parts[2]:
                    right += 1
    
    return right / total

listFiles = ["Kiet_result.txt", "Minh_result.txt", "Nam_result.txt", "Quang_result.txt", "Thien_result.txt"]
f"{check_accu(listFiles):.2%}"

'93.21%'